# Querying Latest Block Height in Libbitcoin

###  Load libraries and header files.
Make sure to adapt information below according to your local installation.

In [3]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib")
#pragma cling load("bitcoin","bitcoin-protocol","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Required header files.
#include <iostream>
#include <string>
#include <bitcoin/bitcoin.hpp> // Libbitcoin-System
#include <bitcoin/protocol.hpp> // Libbitcoin-Protocol

## 1) Setting up the ZMQ requester socket.
We will use a `ZMQ REQ` socket in this example to query the Libbitcoin server for the latest blockheight. The REQ socket is a strictly synchronous, meaning that it must alternate between request and reply, and cannot send multiple requests at once.

<img src="images/zmq_req_rep.jpg" alt="drawing" style="" width="800px"/>

The ZMQ requester socket is setup with the following steps:
* 1.1 Instantiate single ZMQ context object.
* 1.2 Instantiate ZMQ requester socket.
* 1.3 Setup ZMQ Curve authentification.
* 1.4 Connect requester to query service endpoint.


### 1.1 Instantiate single ZMQ context object.
* Only instantiate a single ZMQ context per thread or process. 
* Multiple ZMQ sockets can be created for each context.

In [4]:
bc::protocol::zmq::context my_context(true); // true = started

### 1.2 Instantiate ZMQ requester socket.

In [5]:
bc::protocol::zmq::socket my_requester(
    my_context,
    bc::protocol::zmq::socket::role::requester);
bc::config::endpoint public_endpoint("tcp://testnet1.libbitcoin.net:19081");
bc::code ec_socket; // error code object for socket.

### 1.3 Setup ZMQ Curve authentification.
* [ZMQ Curve](http://api.zeromq.org/4-0:zmq-curve) scheme enables an authenticated and secure connection with service endpoint.
* We will generate a new ZMQ certificate for this query.
* Certificate contains public/private key pair.

In [6]:
// Known server public key for "tcp://testnet1.libbitcoin.net:19081".
bc::config::sodium server_key(")nNv4Ji=CU:}@<LOu-<QvB)b-PIh%PX[)?mH>XAl");

// Generate new certificate: private/public keys.
bc::protocol::zmq::certificate my_certificate;
bc::config::sodium my_private_key = my_certificate.private_key();
bc::config::sodium my_public_key = my_certificate.public_key();

// Prints out base85 representation of generated keys.
std::cout << "Generated Private Key: " << my_private_key.to_string() << std::endl
          << "Generated Public Key: " << my_public_key.to_string()  << std::endl;

my_requester.set_curve_client(server_key);
my_requester.set_certificate(my_certificate);

Generated Private Key: v/rmCvS(!gX5N&8sad]Eiy2[}XoX!ZSj/el{Vx15
Generated Public Key: G..@4*c8OHd*OM46r}+!HcbE-9r0r[SG4=?&tWB<


### 1.4) Connect requester to query service endpoint.
* Successfully connecting the endpoint does not mean a connection is established with the server. 
* It merely means the socket is now ready to establish request/receive communication.
* See zmq_connect in the [ZMQ documentation](http://api.zeromq.org/3-2:zmq-connect).

In [7]:
ec_socket = my_requester.connect(public_endpoint);
std::cout << ec_socket.message() << std::endl;

success


## 2) Send blockheight query & receive reply.

We have know set up our connection objects and are ready to compose messages for our simple <span style="font-size:110%;">&#10122;</span> request and <span style="font-size:110%;">&#10123;</span> response query.

<img src="images/zmq_message.jpg" alt="drawing" style="" width="800px"/>

The following code cells will execute the following:
* 2.1 Build & populate request ZMQ message.
* 2.2 Build response ZMQ message.
* 2.3 Send request, wait/receive reply.

### 2.1 Build & populate request message.
* Each ZMQ message consists of frames, which each hold different message parts.
* For the "block.fetch_last_height" Libbitcoin query, the following message structure is expected:

```
[-- "blockchain.fetch_last_height" --]
[-------- 4-byte message id ---------] (little endian)
[---------- empty payload -----------]
```

In [8]:
std::string command = "blockchain.fetch_last_height";
uint32_t message_id(0);
bc::data_chunk payload({}); 

bc::protocol::zmq::message my_request;
my_request.enqueue(bc::to_chunk(command));
my_request.enqueue(bc::to_chunk(bc::to_little_endian(message_id)));
my_request.enqueue(payload); 


### 2.2 Build response message
* For the "block.fetch_last_height" Libbitcoin server request, the following reply message structure is expected:

```
[---------------- sent message command --------------]
[--------------- sent 4-byte message id -------------] (little endian)
[-- 4-byte error code | 4-byte latest block height --] (little endian)
```

In [9]:
bc::protocol::zmq::message server_response;

// Response Message Data:
std::string my_message_command;
uint32_t my_message_id;  // Little Endian
bc::data_chunk reply_payload; // Little Endian, consists of ec_reply | height
bc::code ec_reply;
uint32_t height;

### 2.3 Send request, wait & receive reply.
* After the request is sent, `zmq_socket.receive()` blocks until a message can be unqueued.
* Upon receiving the reply, the reply payload is parsed with a stream object:
    * [`bc::data_source`](https://github.com/libbitcoin/libbitcoin/blob/master/include/bitcoin/bitcoin/utility/container_source.hpp#L73) is a byte stream class which can be instantiated with a `bc::data_chunk` object.
    * [`bc::istream_reader`](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/utility/istream_reader.hpp#L29) provides methods to parse various data types from a byte stream. 
    
<img src="images/reply_stream_method.jpg" alt="drawing" style="" width="800px"/>

In [10]:
// Submit Query.
//------------------------------------------------------------------------------
ec_socket = my_request.send(my_requester);
std::cout << "-------------------------------------" << std::endl;
std::cout << "Query Error Code: " << ec_socket.message() << std::endl;


// Parse Server Reply.
//------------------------------------------------------------------------------
server_response.receive(my_requester); // Blocking until message available.

my_message_command = server_response.dequeue_text();
server_response.dequeue(my_message_id);
server_response.dequeue(reply_payload);

bc::data_source reply_byte_stream(reply_payload);
bc::istream_reader reply_byte_stream_reader(reply_byte_stream);

ec_reply = reply_byte_stream_reader.read_error_code();
std::cout << "-------------------------------------" << std::endl;
std::cout << "Reply Error Code: " << ec_reply.message() << std::endl;

height = reply_byte_stream_reader.read_4_bytes_little_endian();
std::cout << "Returned Blockheight: " << height << std::endl;

-------------------------------------
Query Error Code: success
-------------------------------------
Reply Error Code: success
Returned Blockheight: 1415055
